In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from glob import glob
import os
import re
import pyemma.msm as msm
import pyemma.coordinates as coor
import numpy as np
import my_network_plot as myplt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import ticker
import operator
import pandas as pd
import mdtraj as md
import matplotlib.image as mpimg 
import pickle


/Users/robertarbon/miniconda3/envs/science/lib/python3.6/site-packages/pyemma/util/log.py:64: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  D = yaml.load(f2)


# Load the data

Below is the parameters cell for papermill parameterisation:

# Identifying interesting time slices


We only want to keep those chunks for which the timescales and corresponding HMM meet certain criteria. 

1. The HMM can't contain absorbing states. 
2. The MSM timescales are all longer than the MSM lag time (so are not spurious). 


## Converge Bayes HMMs for each MLE HMM

In [2]:
from pyemma import config # doctest: +SKIP
config.show_progress_bars = False

In [3]:
bayes_ts = {'traj_num': [], 'time_idx': [], 'mean_ts_ps': []}
bayes_mods = {'traj_num': [], 'time_idx': [], "mod": [] }
root_dir = 'msm_10ps'
lag_ps = 10
for traj_num in [1,3,4,6,7,8,9,10]:
    data_dir = root_dir+'/output_{}'.format(traj_num)
    hmm_paths = glob(os.path.join(data_dir, '*_hmm_obj.pyemma'))
    hmm_idx = [int(re.search('^[0-9]+', os.path.basename(x)).group(0)) for x in hmm_paths]
    dtraj_paths = [os.path.join(data_dir, '{}_dtraj.npy'.format(i)) for i in hmm_idx]
    print(traj_num)
    for i in range(len(hmm_idx)):
        print('\tTrying idx = {}'.format(hmm_idx[i]), end='...')
        # Get HMM parameters: 
        hmm = msm.MaximumLikelihoodHMSM().load(hmm_paths[i])
        nstates = hmm.nstates
        lag = hmm.lag # lag in frames
        dt = np.round(lag_ps/lag, 2) # Timestep in ps. 
        # Check for any silly models
        if np.any(np.diag(hmm.transition_matrix)==1):
            print('Failed - absorbing state :(')
            continue 
        if np.any(hmm.stationary_distribution < 1e-8):
            print('Failed - zero probability state :(')
            continue
        print('OK!')
        # Fit Bayes model
        dtraj = np.load(dtraj_paths[i])
        bhmm = msm.bayesian_hidden_markov_model(dtraj, nstates=nstates, 
                                                lag=lag, nsamples=1000)
        
#         # Get the timescales of off-diagonal elements
#         T = bhmm.sample_mean('transition_matrix')
#         off_diag = T[~np.eye(T.shape[0], dtype='bool')]
#         ts = -lag_ps/np.log(off_diag[off_diag>0])
#         n_ts = ts.shape[0]
        
#         # Record results
#         bayes_ts['traj_num'].extend(np.repeat(traj_num, n_ts))
#         bayes_ts['time_idx'].extend(np.repeat(hmm_idx[i], n_ts))
#         bayes_ts['mean_ts_ps'].extend(ts)

        # Record models
        bayes_mods['time_idx'].append(hmm_idx[i])
        bayes_mods['traj_num'].append(traj_num)
        bayes_mods['mod'].append(bhmm)

1
	Trying idx = 2...

/Users/robertarbon/miniconda3/envs/science/lib/python3.6/site-packages/pyemma/_base/serialization/pickle_extensions.py:52: UserWarning: BLOSC compression filter unavailable. Your resulting file may be large and not optimal to process.
  warnings.warn('BLOSC compression filter unavailable. '


OK!
	Trying idx = 994...Failed - absorbing state :(
	Trying idx = 981...OK!
	Trying idx = 882...OK!
	Trying idx = 32...OK!
	Trying idx = 27...Failed - absorbing state :(
	Trying idx = 127...OK!
	Trying idx = 132...OK!
	Trying idx = 56...OK!
	Trying idx = 143...OK!


/Users/robertarbon/miniconda3/envs/science/lib/python3.6/site-packages/msmtools/analysis/dense/pcca.py:287: ComplexWarning: Casting complex values to real discards the imaginary part
  evecs[:, i] /= math.sqrt(np.dot(evecs[:, i] * pi, evecs[:, i]))


	Trying idx = 156...OK!
	Trying idx = 303...Failed - absorbing state :(
	Trying idx = 963...OK!
	Trying idx = 976...OK!


...

	Trying idx = 194...OK!
	Trying idx = 81...Failed - absorbing state :(
	Trying idx = 112...OK!
	Trying idx = 12...Failed - absorbing state :(
	Trying idx = 626...OK!
	Trying idx = 227...OK!
	Trying idx = 324...Failed - absorbing state :(
	Trying idx = 79...OK!
	Trying idx = 642...OK!
	Trying idx = 186...OK!
	Trying idx = 108...OK!
	Trying idx = 340...OK!
	Trying idx = 935...Failed - absorbing state :(
	Trying idx = 100...OK!
	Trying idx = 348...OK!
	Trying idx = 402...OK!
	Trying idx = 339...OK!
	Trying idx = 14...Failed - absorbing state :(
	Trying idx = 338...OK!


/Users/robertarbon/miniconda3/envs/science/lib/python3.6/site-packages/msmtools/analysis/dense/pcca.py:287: ComplexWarning: Casting complex values to real discards the imaginary part
  evecs[:, i] /= math.sqrt(np.dot(evecs[:, i] * pi, evecs[:, i]))


	Trying idx = 233...OK!
	Trying idx = 192...Failed - absorbing state :(
	Trying idx = 87...Failed - absorbing state :(
	Trying idx = 643...OK!
	Trying idx = 13...Failed - zero probability state :(
	Trying idx = 106...Failed - absorbing state :(
	Trying idx = 404...OK!


/Users/robertarbon/miniconda3/envs/science/lib/python3.6/site-packages/msmtools/analysis/dense/pcca.py:287: ComplexWarning: Casting complex values to real discards the imaginary part
  evecs[:, i] /= math.sqrt(np.dot(evecs[:, i] * pi, evecs[:, i]))


	Trying idx = 322...OK!
	Trying idx = 337...OK!
	Trying idx = 95...OK!


## Save data

In [5]:
# df = pd.DataFrame(bayes_ts)
# df.to_csv('all_timescales.csv')
# pickle.dump(bayes_mods, file=open('all_bayes_mods.pickle', 'wb'))
for i in range(len(bayes_mods['traj_num'])):
    f_name = 'traj_{0}-idx_{1}_bhmm.p'.format(bayes_mods['traj_num'][i], bayes_mods['time_idx'][i])
    pickle.dump(obj=bayes_mods['mod'][i], file=open(f_name, 'wb'))